# TODO

- what to do with dates without years? -> NaN probably?
- drop original column? 


# Imports

In [2]:
# Imports
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import re 
from math import isnan

pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_seq_items', None)

In [3]:
# dtype={'type': str} prevents being confused with data type for large data sets
train = pd.read_csv('data/train.csv', index_col='id', dtype={'type': str})
test = pd.read_csv('data/test.csv', index_col='id', dtype={'type': str})
train_translated = pd.read_csv('data/train_translated.csv', dtype={'type': str})
test_translated = pd.read_csv('data/test_translated.csv', index_col='id', dtype={'type': str})
combined_data = pd.read_csv('data/combined_data.csv', index_col='id', dtype={'type': str})
combined_data_translated = pd.read_csv('data/combined_data_translated.csv', index_col='id', dtype={'type': str})
combined_data_fully_translated = pd.read_csv('data/combined_data_fully_translated.csv', index_col='id', dtype={'type': str})
prep = pd.read_csv('data/prep.csv', index_col='id', dtype={'type': str})

C:\Users\flori\AppData\Local\Temp\ipykernel_8960\1699356385.py:9: DtypeWarning: Columns (13,266) have mixed types. Specify dtype option on import or set low_memory=False.
  prep = pd.read_csv('data/prep.csv', index_col='id', dtype={'type': str})


# Formatting

The following columns need to be formatted:
- start 
- end

In [4]:
data = prep.copy()
data.head()

,full_nr,name,ks,commentary,event_type,location,start,end,before_Christ,country_and_unit,participants_role,participant,parish,text,class,parameter,unit,value,museum_abbr,musealia_mark,musealia_seria_nr,musealia_queue_nr,musealia_additional_nr,collection_mark,collection_queue_nr,collection_additional_nr,element_count,legend,is_original,initial_info,damages,state,color,additional_text,type,source,city_municipality,country,material_Polish,material_RC Photo Paper,material_a pearl,material_acetate cellulose film,material_albumen paper,material_albumin paper,material_aluminium,material_amber,material_artificial fiber material,material_artificial leather,material_artificial material,material_atlas,material_ballpoint pen ink,material_birch,material_bone,material_brass,material_brocade (clothing variety),material_bronze,material_canvas,material_canvas (type of cloth),material_cardboard,material_cast iron,material_celluloid,material_ceramics,material_chalk,material_chamois leather,material_chamotte,material_charcoal,material_chromogen emulsion,material_chromogen paper,material_clay,material_clothing variety,material_collodion paper,material_colloid paper,material_colour,material_copper,material_cotton,material_crepe,material_crepe (cloth type),material_crystal,material_diffusion paper,material_email,material_emulsion,material_enamel paint,material_faience,material_feather,material_film,material_film (material),material_flint,material_from the bat,material_glass,material_gold,material_granite,material_graphite,material_gypsum,material_handmade paper,material_ink,material_iron,material_kalka,material_knitwear,material_leotard (type of clothing),material_linen,material_mascara,material_metal,material_metal fibers,material_moire (clothing variety),material_movie,material_nan,material_newsprint,material_nitrocellulose film,material_nut,material_oil paint,material_organic matter,material_paper,material_papier mache,material_photo emulsion,material_photo material,material_photo paper,material_photo plate,material_photographic material,material_plastic,material_plastic mass,material_plywood,material_porcelain,material_printing ink,material_quartz,material_rubber,material_salt paper,material_silk,material_silver,material_silver gelatin emulsion,material_silver gelatin paper,material_skin,material_slate,material_stone,material_synthetic fibers,material_synthetic material,material_tempera,material_textile,material_tin,material_trillion,material_watercolor paint,material_wax,material_white metal,material_wire,material_wood,material_wood material,material_wooden board,material_wool,material_yarn,technique_(close/together) sewing,technique_addition,technique_aquatint,technique_ballpoint pen,technique_binding techniques,technique_black and white photo,technique_black and white photography,technique_bronzing,technique_brushing,technique_burning,technique_carving,technique_casting,technique_chalk,technique_charcoal,technique_chromogen procedure,technique_collage,technique_collotype,technique_color photo,technique_color photography,technique_colored chalk,technique_colored pencil,technique_coloring,technique_copper engraving,technique_copying,technique_crayon drawing,technique_croaking,technique_crocheting,technique_cutting,technique_cuttlefish,technique_daguerreotype,technique_digital photography,technique_digital printing,technique_drawing,technique_electronic imaging,technique_embroidery,technique_enamelling,technique_engraving,technique_etching,technique_felt tip pen,technique_glass technology,technique_gouache,technique_graphics,technique_graphite,technique_handicraft,technique_handwriting,technique_ink,technique_ink drawing,technique_intarsia,technique_kiln ceramics,technique_knitting,technique_letterpress with raster cliché,technique_linocut,technique_lithography,technique_manuscript,technique_marker,technique_mascara,technique_mezzotinto,technique_mixed media,technique_modeling,technique_molding,technique_monotypy,technique_nan,technique_offset p

In [5]:
data['start'].value_counts()

1900          148
1970          130
1944          120
1948          117
1947          112
1960          109
1945          104
1958           98
1969           96
1801           94
1959           93
1967           91
1980           88
1965           83
1964           83
1971           83
1949           81
1961           81
1989           79
1950           79
1973           78
1940           77
1938           75
1957           74
1963           74
1966           74
1701           73
1930           73
1954           73
1951           72
1946           71
1939           71
1968           70
1920           67
1952           67
1937           63
1936           63
1953           62
1910           62
1977           61
1935           61
1500           57
1955           56
1974           55
1962           54
1975           53
1934           51
1978           50
1986           49
1972           48
1976           47
1987           46
1981           44
1923           43
1979           43
1931      

In [6]:
data['end'].value_counts()

1910          128
1900          107
1800           71
1940           63
1920           50
1930           40
18.04.1974     39
1980           37
1939           37
2000           34
21.11.1975     31
1550           28
29.06.1947     25
29.06.1969     24
1948           23
21.07.1960     22
23.07.1950     22
1100           21
1990           19
22.07.1955     16
1918           15
1945           15
1979           15
1965           14
1924           14
1056           13
1039           13
1935           12
30.09.1983     11
1929           11
1700           11
1995           11
1959           11
1050           11
1960           11
20.07.1975     10
25.06.1933     10
1944           10
1531           10
1942            9
1928            9
1527            9
1967            9
1984            9
1966            9
1600            9
1943            8
27.06.1961      8
1955            8
1970            8
27.08.1961      8
1931            8
1971            8
18.07.1965      8
01.07.1973      8
1003      

In [7]:
#sortedStart = sorted(data['start'].unique())
#print (sortedStart)
print(data['start'].unique())

['1979' '1938' nan ... 'dets.57' '20.03.1983' '03.03.2000']


In [8]:
#groups the year into its own column with the complete number
def year_Grouping(x):
    xStr = str(x)
    if '.' in xStr:
        xStr = xStr.split('.')
        xStr = xStr[xStr.__len__()-1]
        if xStr.__len__() == 2:
            xStr = '19' + xStr
        elif xStr == '':
            return None
        return xStr
    else:
        return xStr

#groups the month into its own column with numbers between 1-12
def month_Grouping(x):
    xStr = str(x)
    if '.' in xStr:
        xStr = xStr.split('.')
        if xStr[0].__len__() <= 2 and xStr[1].__len__() <= 2:
            if xStr[1] == '':
                return None
            elif xStr[1].__len__() == 1:
                return '0' + xStr[1]
            return xStr[1]
        xStr = xStr[0]
        if xStr == 'jaan':
            return '01'
        elif xStr == 'veebr':
            return '02'
        if xStr == 'märts':
            return '03'
        elif xStr == 'apr':
            return '04'
        elif xStr == 'mai':
            return '05'
        elif xStr == 'juuni':
            return '06'
        elif xStr == 'juuli':
            return '07'
        elif xStr == 'aug':
            return '08'
        elif xStr == 'sept':
            return '09'
        elif xStr == 'okt':
            return '10'
        elif xStr == 'nov':
            return '11'
        elif xStr == 'dets':
            return '12'
        elif xStr.__len__() == 1:
            return '0' + xStr
        return xStr
    else:
        return None

#groups the day into its own column with numbers between 1-31
def day_Grouping(x):
    xStr = str(x)
    if '.' in xStr:
        xStr = xStr.split('.')
        if xStr[0].__len__() <= 2 and xStr[1].__len__() <= 2:
            return xStr[0]
        return None
    else:
        return None

#grouping applied to the dataframe
data['startYear'] = data['start'].apply(year_Grouping)
data['startMonth'] = data['start'].apply(month_Grouping)
data['startDay'] = data['start'].apply(day_Grouping)
        
data['endYear'] = data['end'].apply(year_Grouping)
data['endMonth'] = data['end'].apply(month_Grouping)
data['endDay'] = data['end'].apply(day_Grouping)


# Testing

In [9]:
print(data[['start','startYear', 'startMonth', 'startDay','end', 'endYear', 'endMonth', 'endDay']].head(30))

              start startYear startMonth startDay   end endYear endMonth  \
id                                                                         
232170         1979      1979       None     None   NaN     nan     None   
2251378        1938      1938       None     None   NaN     nan     None   
2070466         NaN       nan       None     None   NaN     nan     None   
4085096  18.09.1969      1969         09       18   NaN     nan     None   
2697904         NaN       nan       None     None   NaN     nan     None   
2521183        1500      1500       None     None  1550    1550     None   
224392         1978      1978       None     None   NaN     nan     None   
2111663         NaN       nan       None     None   NaN     nan     None   
2746349         NaN       nan       None     None   NaN     nan     None   
198694          NaN       nan       None     None   NaN     nan     None   
2748234      mai.44      1944         05     None   NaN     nan     None   
2352234     

In [10]:
#save 
data.to_csv('data/prep.csv')